In [1]:
import pandas as pd

from utils.funcs import extract_substring_between_parentheses

/home/ytian/anaconda3/envs/receptorgpt/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
mcpas = pd.read_csv("data/McPAS-TCR.csv", encoding="windows-1252")
print(mcpas.shape)
mcpas.head()

(39985, 29)


/tmp/ipykernel_27278/3536489671.py:1: DtypeWarning: Columns (13,17,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  mcpas = pd.read_csv("data/McPAS-TCR.csv", encoding="windows-1252")


,CDR3.alpha.aa,CDR3.beta.aa,Species,Category,Pathology,Pathology.Mesh.ID,Additional.study.details,Antigen.identification.method,Single.cell,NGS,...,TRAV,TRAJ,TRBV,TRBD,TRBJ,Reconstructed.J.annotation,CDR3.beta.nt,Mouse.strain,PubMed.ID,Remarks
0,NaN,CASSDAGANTEVF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NaN,2.1,No,No,...,NaN,NaN,TRBV8-1,NaN,TRBJ1-1,No,NaN,P14,1716213,NaN
1,NaN,CASSDAGAYAEQF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NaN,2.1,No,No,...,NaN,NaN,TRBV8-1,NaN,TRBJ2-1,No,NaN,P14,1716213,NaN
2,NaN,CASSDAGGAAEVF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NaN,2.1,No,No,...,NaN,NaN,TRBV8-3,NaN,TRBJ1-1,No,NaN,P14,1716213,NaN
3,NaN,CASSDAGHSPLYF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NaN,2.1,No,No,...,NaN,NaN,TRBV8-1,NaN,TRBJ1-6,No,NaN,P14,1716213,NaN
4,NaN,CASSDAWGGAEQYF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,NaN,2.1,No,No,...,NaN,NaN,TRBV8-3,NaN,TRBJ2-6,No,NaN,P14,1716213,NaN


In [3]:
# beta chain only
mcpas = mcpas[
    [
        "Species",
        "Epitope.peptide",
        "Antigen.protein",
        "Pathology",
        "CDR3.beta.aa",
        "TRBV",
        "TRBJ",
        "PubMed.ID",
    ]
]

In [4]:
# drop TCRs with missing information
mcpas = mcpas.dropna()
print(mcpas.shape)
mcpas.head()

(12730, 8)


,Species,Epitope.peptide,Antigen.protein,Pathology,CDR3.beta.aa,TRBV,TRBJ,PubMed.ID
0,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,Lymphocytic choriomeningitis virus (LCMV),CASSDAGANTEVF,TRBV8-1,TRBJ1-1,1716213
1,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,Lymphocytic choriomeningitis virus (LCMV),CASSDAGAYAEQF,TRBV8-1,TRBJ2-1,1716213
2,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,Lymphocytic choriomeningitis virus (LCMV),CASSDAGGAAEVF,TRBV8-3,TRBJ1-1,1716213
3,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,Lymphocytic choriomeningitis virus (LCMV),CASSDAGHSPLYF,TRBV8-1,TRBJ1-6,1716213
4,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,Lymphocytic choriomeningitis virus (LCMV),CASSDAWGGAEQYF,TRBV8-3,TRBJ2-6,1716213


In [5]:
# to make it consistent with vdjdb
mcpas["PubMed.ID"] = mcpas["PubMed.ID"].map(lambda x: f"PMID:{x}")

mcpas["Pathology"] = mcpas["Pathology"].map(
    lambda s: extract_substring_between_parentheses(s)
)
mcpas.head()

,Species,Epitope.peptide,Antigen.protein,Pathology,CDR3.beta.aa,TRBV,TRBJ,PubMed.ID
0,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAGANTEVF,TRBV8-1,TRBJ1-1,PMID:1716213
1,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAGAYAEQF,TRBV8-1,TRBJ2-1,PMID:1716213
2,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAGGAAEVF,TRBV8-3,TRBJ1-1,PMID:1716213
3,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAGHSPLYF,TRBV8-1,TRBJ1-6,PMID:1716213
4,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAWGGAEQYF,TRBV8-3,TRBJ2-6,PMID:1716213


In [6]:
vdjdb = pd.read_csv("data/vdjdb.txt", sep="\t")
print(vdjdb.shape)
vdjdb.head()

(92771, 21)


,complex.id,gene,cdr3,v.segm,j.segm,species,mhc.a,mhc.b,mhc.class,antigen.epitope,...,antigen.species,reference.id,method,meta,cdr3fix,vdjdb.score,web.method,web.method.seq,web.cdr3fix.nc,web.cdr3fix.unmp
0,1,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CIVRAPGRADMRF"", ""cdr3_old"": ""CIVRAPG...",2,sort,sanger,no,no
1,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2,sort,sanger,no,no
2,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEAGQGFFSNQPQHF"", ""cdr3_old"": ""C...",2,sort,sanger,no,no
3,2,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAVPSGAGSYQLTF"", ""cdr3_old"": ""CAVPSG...",2,sort,sanger,no,no
4,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2,sort,sanger,no,no


In [7]:
vdjdb["species"].value_counts()

species
HomoSapiens      85329
MusMusculus       5323
MacacaMulatta     2119
Name: count, dtype: int64

In [8]:
# remove monkeys
vdjdb = vdjdb.loc[vdjdb["species"] != "MacacaMulatta"]

# rename
vdjdb["species"] = vdjdb["species"].map(
    {"HomoSapiens": "Human", "MusMusculus": "Mouse"}
)

In [9]:
# TRB only
vdjdb = vdjdb.loc[vdjdb["gene"] == "TRB"]
vdjdb.head()

,complex.id,gene,cdr3,v.segm,j.segm,species,mhc.a,mhc.b,mhc.class,antigen.epitope,...,antigen.species,reference.id,method,meta,cdr3fix,vdjdb.score,web.method,web.method.seq,web.cdr3fix.nc,web.cdr3fix.unmp
1,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,Human,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2,sort,sanger,no,no
2,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,Human,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEAGQGFFSNQPQHF"", ""cdr3_old"": ""C...",2,sort,sanger,no,no
4,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,Human,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2,sort,sanger,no,no
6,3,TRB,CASSYEPGQVSHYSNQPQHF,TRBV13*01,TRBJ1-5*01,Human,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYEPGQVSHYSNQPQHF"", ""cdr3_old"": ""...",2,sort,sanger,no,no
8,4,TRB,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,Human,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSALASLNEQFF"", ""cdr3_old"": ""CASSAL...",2,sort,sanger,no,no


In [10]:
# database name
mcpas["Database"] = "McPAS-TCR"
vdjdb["Database"] = "VDJdb"

# rename the columns to make the consistent
mcpas = mcpas.rename(
    columns={
        "Epitope.peptide": "Antigen Epitope",
        "Antigen.protein": "Antigen Protein",
        "Pathology": "Antigen Source",
        "PubMed.ID": "Reference",
    }
)

vdjdb = vdjdb.rename(
    columns={
        "species": "Species",
        "antigen.epitope": "Antigen Epitope",
        "antigen.gene": "Antigen Protein",
        "antigen.species": "Antigen Source",
        "reference.id": "Reference",
        "cdr3": "CDR3.beta.aa",
        "v.segm": "TRBV",
        "j.segm": "TRBJ",
    }
)

In [11]:
cols = mcpas.columns.to_list()
vdjdb = vdjdb[cols]
vdjdb = vdjdb.dropna()
print(vdjdb.shape)
vdjdb.head()

(51730, 9)


,Species,Antigen Epitope,Antigen Protein,Antigen Source,CDR3.beta.aa,TRBV,TRBJ,Reference,Database
1,Human,FLKEKGGL,Nef,HIV-1,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,PMID:15596521,VDJdb
2,Human,FLKEKGGL,Nef,HIV-1,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,PMID:15596521,VDJdb
4,Human,FLKEKGGL,Nef,HIV-1,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,PMID:15596521,VDJdb
6,Human,FLKEKGGL,Nef,HIV-1,CASSYEPGQVSHYSNQPQHF,TRBV13*01,TRBJ1-5*01,PMID:15596521,VDJdb
8,Human,FLKEKGGL,Nef,HIV-1,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,PMID:15596521,VDJdb


In [12]:
# IEDB
iedb = pd.read_csv("data/iedb_tcr_full_v3.csv", header=1)
print(iedb.shape)
iedb.head()

(218277, 69)


/tmp/ipykernel_27278/933994250.py:2: DtypeWarning: Columns (2,10,11,12,13,14,15,16,17,18,19,20,21,22,23,28,29,34,35,40,41,42,43,44,45,46,47,48,49,50,51,52,57,58,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  iedb = pd.read_csv("data/iedb_tcr_full_v3.csv", header=1)


,Group IRI,IEDB Receptor ID,Reference Name,Type,IEDB IRI,IEDB IRI.1,Name,Source Molecule,Source Organism,Type.1,...,CDR1 Start Curated.1,CDR1 End Curated.1,CDR1 Start Calculated.1,CDR1 End Calculated.1,CDR2 Curated.1,CDR2 Calculated.1,CDR2 Start Curated.1,CDR2 End Curated.1,CDR2 Start Calculated.1,CDR2 End Calculated.1
0,http://www.iedb.org/receptor/47,57,KK50.4,alphabeta,http://www.iedb.org/reference/1004539,http://www.iedb.org/epitope/69921,VMAPRTLIL,"HLA class I histocompatibility antigen, Cw-3 a...",Homo sapiens (human),T cell,...,NaN,NaN,25.0,29.0,NaN,FVKESK,NaN,NaN,47.0,52.0
1,http://www.iedb.org/receptor/47,57,KK50.4,alphabeta,http://www.iedb.org/reference/1004539,http://www.iedb.org/epitope/69921,VMAPRTLIL,"HLA class I histocompatibility antigen, Cw-3 a...",Homo sapiens (human),T cell,...,NaN,NaN,25.0,29.0,NaN,FVKESK,NaN,NaN,47.0,52.0
2,http://www.iedb.org/receptor/8493,58,LC13,alphabeta,http://www.iedb.org/reference/1004580,http://www.iedb.org/epitope/16878,FLRGRAYGL,nuclear antigen EBNA-3,Human herpesvirus 4 (Epstein Barr virus),T cell,...,NaN,NaN,25.0,29.0,NaN,FQNEAQ,NaN,NaN,47.0,52.0
3,http://www.iedb.org/receptor/8493,58,LC13,alphabeta,http://www.iedb.org/reference/1004580,http://www.iedb.org/epitope/144889,FLRGRFYGL,NaN,NaN,T cell,...,NaN,NaN,25.0,29.0,NaN,FQNEAQ,NaN,NaN,47.0,52.0
4,http://www.iedb.org/receptor/8493,58,LC13,alphabeta,http://www.iedb.org/reference/1017865,http://www.iedb.org/epitope/142137,EEYLQAFTY,ATP-binding cassette sub-family D member 3,Homo sapiens (human),T cell,...,NaN,NaN,25.0,29.0,NaN,FQNEAQ,NaN,NaN,47.0,52.0


In [13]:
# CEDAR
cedar = pd.read_csv("data/cedar_tcr_full_v3.csv", header=1)
print(cedar.shape)
cedar.head()

(102311, 70)


/tmp/ipykernel_27278/1237765955.py:2: DtypeWarning: Columns (2,10,12,13,14,15,16,17,19,20,21,22,23,24,29,30,35,36,41,42,43,44,45,46,47,48,49,50,51,52,53,58,59,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  cedar = pd.read_csv("data/cedar_tcr_full_v3.csv", header=1)


,Group IRI,IEDB Receptor ID,Reference Name,Type,IEDB IRI,IEDB IRI.1,Name,Source Molecule,Source Organism,Type.1,...,CDR1 Start Curated.1,CDR1 End Curated.1,CDR1 Start Calculated.1,CDR1 End Calculated.1,CDR2 Curated.1,CDR2 Calculated.1,CDR2 Start Curated.1,CDR2 End Curated.1,CDR2 Start Calculated.1,CDR2 End Calculated.1
0,http://www.iedb.org/receptor/47,57,KK50.4,alphabeta,http://www.iedb.org/reference/1004539,http://www.iedb.org/epitope/69921,VMAPRTLIL,"HLA class I histocompatibility antigen, Cw-3 a...",Homo sapiens (human),T cell,...,NaN,NaN,25.0,29.0,NaN,FVKESK,NaN,NaN,47.0,52.0
1,http://www.iedb.org/receptor/47,57,KK50.4,alphabeta,http://www.iedb.org/reference/1004539,http://www.iedb.org/epitope/69921,VMAPRTLIL,"HLA class I histocompatibility antigen, Cw-3 a...",Homo sapiens (human),T cell,...,NaN,NaN,25.0,29.0,NaN,FVKESK,NaN,NaN,47.0,52.0
2,http://www.iedb.org/receptor/8493,58,LC13,alphabeta,http://www.iedb.org/reference/1004580,http://www.iedb.org/epitope/16878,FLRGRAYGL,nuclear antigen EBNA-3,Human herpesvirus 4 (Epstein Barr virus),T cell,...,NaN,NaN,25.0,29.0,NaN,FQNEAQ,NaN,NaN,47.0,52.0
3,http://www.iedb.org/receptor/8493,58,LC13,alphabeta,http://www.iedb.org/reference/1004580,http://www.iedb.org/epitope/144889,FLRGRFYGL,NaN,NaN,T cell,...,NaN,NaN,25.0,29.0,NaN,FQNEAQ,NaN,NaN,47.0,52.0
4,http://www.iedb.org/receptor/8493,58,LC13,alphabeta,http://www.iedb.org/reference/1017865,http://www.iedb.org/epitope/156273,EEYLKAWTF,NaN,NaN,T cell,...,NaN,NaN,25.0,29.0,NaN,FQNEAQ,NaN,NaN,47.0,52.0


In [14]:
def process_iedb(df, db_name):
    df = df.copy()
    # the two chain should have the same species
    df = df.loc[df["Organism IRI"] == df["Organism IRI.1"]]

    df["Species"] = df["Organism IRI"].map(
        {
            "http://purl.obolibrary.org/obo/NCBITaxon_9606": "Human",
            "http://purl.obolibrary.org/obo/NCBITaxon_10090": "Mouse",
        }
    )

    # only keep human and mouse
    df = df.loc[df["Species"].isin(["Human", "Mouse"])]

    # use curated data
    df = df.rename(
        columns={
            "Name": "Antigen Epitope",
            "Source Molecule": "Antigen Protein",
            "Source Organism": "Antigen Source",
            "IEDB IRI": "Reference",
            "CDR3 Curated.1": "CDR3.beta.aa",
            "Curated V Gene.1": "TRBV",
            "Curated J Gene.1": "TRBJ",
        }
    )

    df["Database"] = db_name
    df = df[cols]
    df = df.dropna()
    df["Antigen Source"] = df["Antigen Source"].map(
        lambda s: extract_substring_between_parentheses(s)
    )
    return df

In [15]:
iedb = process_iedb(iedb, "IEDB")
cedar = process_iedb(cedar, "CEDAR")

In [16]:
iedb.head()

,Species,Antigen Epitope,Antigen Protein,Antigen Source,CDR3.beta.aa,TRBV,TRBJ,Reference,Database
0,Human,VMAPRTLIL,"HLA class I histocompatibility antigen, Cw-3 a...",human,ASSQDRDTQY,TRBV14*01,TRBJ2-3*01,http://www.iedb.org/reference/1004539,IEDB
36,Human,CINGVCWTV,polyprotein,Hepacivirus hominis,ASSQGGDRGDPGDGYT,TCRBV5-06,TCRBJ1-02,http://www.iedb.org/reference/1032053,IEDB
109,Human,SLLMWITQC,NY-ESO-1 protein,human,CASSYVGNTGELFF,TRBV6-5*01,TRBJ2-2*01,http://www.iedb.org/reference/1012513,IEDB
200,Human,GLCTLVAML,BMLF1 protein,Epstein Barr virus,SARDGTGNGYT,TCRBV20-1,TCRBJ1-2,http://www.iedb.org/reference/1021176,IEDB
202,Human,LSRFSWGAEGQRPGFGYGG,MBP protein,human,SARGGSYNSPLH,TRBV20-1*01,TRBJ1-6*01,http://www.iedb.org/reference/365,IEDB


In [17]:
cedar.head()

,Species,Antigen Epitope,Antigen Protein,Antigen Source,CDR3.beta.aa,TRBV,TRBJ,Reference,Database
0,Human,VMAPRTLIL,"HLA class I histocompatibility antigen, Cw-3 a...",human,ASSQDRDTQY,TRBV14*01,TRBJ2-3*01,http://www.iedb.org/reference/1004539,CEDAR
36,Human,CINGVCWTV,polyprotein,Hepatitis C virus,ASSQGGDRGDPGDGYT,TCRBV5-06,TCRBJ1-02,http://www.iedb.org/reference/1032053,CEDAR
88,Human,SLLMWITQC,NY-ESO-1 protein,human,CASSYVGNTGELFF,TRBV6-5*01,TRBJ2-2*01,http://www.iedb.org/reference/1012513,CEDAR
127,Human,GLCTLVAML,BMLF1 protein,Epstein Barr virus,SARDGTGNGYT,TCRBV20-1,TCRBJ1-2,http://www.iedb.org/reference/1021176,CEDAR
167,Human,AAGIGILTV,Melanoma antigen recognized by T-cells 1,human,CAISEVGVGQPQHF,TRBV10-3,TRBJ1-5*01,http://www.iedb.org/reference/1012513,CEDAR


In [18]:
tcrs = pd.concat([mcpas, vdjdb, iedb, cedar])
tcrs.shape

(81288, 9)

In [19]:
# remove duplicates
tcrs = tcrs.drop_duplicates(subset="CDR3.beta.aa", ignore_index=True)
print(tcrs.shape)
tcrs.head()

(48540, 9)


,Species,Antigen Epitope,Antigen Protein,Antigen Source,CDR3.beta.aa,TRBV,TRBJ,Reference,Database
0,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAGANTEVF,TRBV8-1,TRBJ1-1,PMID:1716213,McPAS-TCR
1,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAGAYAEQF,TRBV8-1,TRBJ2-1,PMID:1716213,McPAS-TCR
2,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAGGAAEVF,TRBV8-3,TRBJ1-1,PMID:1716213,McPAS-TCR
3,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAGHSPLYF,TRBV8-1,TRBJ1-6,PMID:1716213,McPAS-TCR
4,Mouse,IKAVYNFATCG,Pre-glycoprotein polyprotein GP complex,LCMV,CASSDAWGGAEQYF,TRBV8-3,TRBJ2-6,PMID:1716213,McPAS-TCR


In [20]:
tcrs.to_parquet("data/antigen_specific_tcrs.parquet")